In [ ]:
# testing vertex ai vector search for embedding & storage, 10/07 sdrfsfsdfdsf sdfsfsd

### TODO


*   Tune chunking parameters
*   Choose embedding model
*   Implement Vector Search



In [1]:
!pip install langchain-google-vertexai
!pip install langchain
!pip install qdrant-client[fastembed]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.8.0
    Uninstalling google-cloud-storage-2.8.0:
      Successfully uninstalled google-cloud-storage-2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.4 MB/s eta 0:00:00


In [2]:
from datetime import datetime
PROJECT_ID = "cs-crimsonchat"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
UID = datetime.now().strftime("%m%d%H%M")
bucket_name = f"{PROJECT_ID}-vertexai-embedding-ms2-{UID}"

In [4]:
from langchain_google_vertexai import VertexAIEmbeddings
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)
# Initialize the a specific Embeddings Model version
embeddings_model = VertexAIEmbeddings(model_name="text-embedding-004") #can change embedding model, 768D

# Vertex AI embedding modelhttps://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/hybrid-search.ipynb
def get_dense_embedding(text):
    return embeddings_model.embed_query(text)

 default, the text embeddings API has a "request per minute" quota set to 60 for new Cloud projects and 600 for projects with usage history (see Quotas and limits to check the latest quota value for base_model:textembedding-gecko). So, rather than using the function directly, you may want to define a wrapper like below to limit under 10 calls per second, and pass 5 texts each time.
 https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/intro-textemb-vectorsearch.ipynb

In [5]:
# Chunking mechanism
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len,
)

In [6]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Create a sample DataFrame
df = pd.DataFrame({
    'doc_id': [i for i in range(5)],
    'text': [
        "This is a short text.",
        "This is a bit longer text that might need chunking.",
        "This is an even longer text that will definitely need chunking. It contains multiple sentences and more information.",
        "This is a very long text that will be split into multiple chunks. It contains a lot of information and spans multiple sentences. We'll use this to demonstrate how chunking works with metadata.",
        "This is the longest text in our dataset. It contains many sentences and a lot of information. This text is designed to be split into several chunks to show how the chunking process works and how metadata is attached to each chunk.",
        "Dinosaurs were a diverse group of reptiles that dominated the Earth for over 160 million years during the Mesozoic Era. They ranged from small, agile creatures to massive, lumbering giants, and evolved into a wide variety of species adapted to different environments and ecological niches. Some were fierce predators, while others were peaceful plant-eaters. Dinosaurs' reign came to an abrupt end about 66 million years ago when a catastrophic asteroid impact triggered a mass extinction event. However, their legacy lives on in modern birds, which are considered living dinosaurs by scientists. The study of dinosaurs continues to captivate both researchers and the public, offering insights into Earth's prehistoric past and the processes of evolution.",
        "Dinosaur candy is a playful confectionery that brings prehistoric fun to the world of sweets. These treats often come in the shape of various dinosaur species, appealing to both children and adults with a fondness for paleontology. From gummy T-Rexes to chocolate triceratops, dinosaur candies combine the excitement of ancient creatures with the joy of sugary snacks."
    ],
    'url': [f"https://example.com/text_{i}" for i in range(5)],
    'scraped_datetime': [datetime.now() - timedelta(days=i) for i in range(5)]
})


documents_embedded = []
for i in range(len(df)):
    id = df.doc_id[i]
    text = df.text[i]
    url = df.url[i]
    scraped_datetime = df.scraped_datetime[i]

    # Split the text into chunks
    chunks = text_splitter.split_text(text)

    # Process each chunk
    for chunk_id, chunk in enumerate(chunks):
        dense_embedding = get_dense_embedding(chunk)
        documents_embedded.append({
            "id": f"{id}_{chunk_id}",  # Unique ID for each chunk
            "text": chunk,
            "url": url,
            "scraped_datetime": scraped_datetime,
            "embedding": dense_embedding,
            "original_text_id": id,  # Keep track of which original text this chunk came from
            "chunk_id": chunk_id  # Keep track of the chunk number within the original text
        })

In [7]:
print(len(documents_embedded[0]["embedding"]))
print(len(documents_embedded))

768
17


# QDRANT

In [8]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    url="https://1494f517-c19c-490b-8a4e-43ff3b02bbb7.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key="5qJBIKdEycPYlWfaDiAwd-1Hz2z88qaBsSV_UAa4AljpqWpWGzmRTg",
)

print(qdrant_client.get_collections())

collections=[]


In [20]:
list_collections = qdrant_client.get_collections()
collection_name = list_collections.collections[0].name
print("collection name: ", collection_name)

collection name:  ms2-testing


In [32]:
import uuid
from qdrant_client import QdrantClient, models
from qdrant_client.http import exceptions as qdrant_exceptions

def qdrant_transform_for_upsert(documents_embedded, collection_name):
    points = []
    for doc in documents_embedded:
        point = models.PointStruct(
            id=str(uuid.uuid4()),
            payload={
                "text": doc["text"],
                "url": doc["url"],
                "scraped_datetime": doc["scraped_datetime"],
                "original_text_id": int(doc["original_text_id"]),
                "chunk_id": int(doc["chunk_id"])
            },
            vector=[float(v) for v in doc["embedding"]]
        )
        points.append(point)

    return {
        "collection_name": collection_name,
        "points": points
    }

def qdrant_upsert(qdrant_client, qdrant_upsert_data):
    try:
        result = qdrant_client.upsert(**qdrant_upsert_data)
        return result
    except qdrant_exceptions.UnexpectedResponse as e:
        raise Exception(f"Unexpected response from Qdrant: {e}")
    except qdrant_exceptions.NetworkError as e:
        raise Exception(f"Network error occurred while communicating with Qdrant: {e}")
    except Exception as e:
        raise Exception(f"An unexpected error occurred during Qdrant upsert: {e}")

# Usage
try:
    collection_name = collection_name
    qdrant_upsert_data = qdrant_transform_for_upsert(documents_embedded, collection_name)
    result = qdrant_upsert(qdrant_client, qdrant_upsert_data)
    print(result)
    # Process the result if needed
except Exception as e:
    raise Exception(f"Failed to upsert data to Qdrant: {e}")

print(result)

In [33]:
print(result)

operation_id=5 status=<UpdateStatus.COMPLETED: 'completed'>


# CLEAN UP


# Google Vector Search Archive Implementation
# Deploying embeddings into Index (~vector database) for Vector Search

In [ ]:
# Create Google Cloud Bucket
from google.cloud import storage

# Create the new bucket
client = storage.Client()
bucket = client.create_bucket(bucket_name, location=LOCATION)
print(f"Bucket {bucket.name} created.")

In [ ]:
# Upload the file to the new bucket
file_name = json_documents_embedded_filename
blob = bucket.blob(file_name)
blob.upload_from_filename(file_name, content_type='application/json')
print(f"File {file_name} uploaded to {bucket.name} in {LOCATION}")

File documents_embedded_10081802.json uploaded to cs-crimsonchat-vertexai-embedding-ms2-10081802 in us-central1


In [ ]:
# sanity check
from google.cloud import storage

def get_blob_content(bucket_name, source_blob_name):
    """Retrieves a blob's content from the bucket."""
    print(f"Requesting info on file {source_blob_name} from bucket {bucket_name}")
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)

    # Fetch the blob's metadata ## NOTE: here not downloading blob for content
    blob.reload()
    print(f"Content type: {blob.content_type}")
    print(f"Size: {blob.size} bytes")


bucket_name = bucket_name
source_blob_name = file_name
blob_content = get_blob_content(bucket_name, source_blob_name)

Requesting info on file documents_embedded_10081802.json from bucket cs-crimsonchat-vertexai-embedding-ms2-10081802
Content type: application/json
Size: 178115 bytes


Vector search initiation
ref: https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/vector-search-quickstart.ipynb

In [ ]:
# init the aiplatform package
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
BUCKET_URI = f"gs://cs-crimsonchat-vertexai-embedding-ms2-10081802"
INDEX_NAME = f"vertexai_embedding_ms2_{UID}"
DEPLOYED_INDEX_ID = INDEX_NAME # Index & Index Endpoints now have same name, but not a good practice

In [ ]:
# create Index
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name= INDEX_NAME,
    contents_delta_uri=BUCKET_URI,
    dimensions=768, # synchronize if changed embedding model
    approximate_neighbors_count=10, # hyperparameter
)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Creating MatchingEngineIndex
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Create MatchingEngineIndex backing LRO: projects/692586115434/locations/us-central1/indexes/5743897122013446144/operations/7383313661047078912
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:MatchingEngineIndex created. Resource name: projects/692586115434/locations/us-central1/indexes/5743897122013446144
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:To use this MatchingEngineIndex in another session:
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:index = aiplatform.MatchingEngineIndex('projects/692586115434/locations/us-central1/indexes/5743897122013446144')


In [ ]:
# create IndexEndpoint, a server instance accepting query requests for your Index
# "public" is only for members of Project
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=INDEX_NAME, public_endpoint_enabled=True
)

# deploy the Index to the Index Endpoint
my_index_endpoint.deploy_index(index=my_index, deployed_index_id=DEPLOYED_INDEX_ID)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Creating MatchingEngineIndexEndpoint
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Create MatchingEngineIndexEndpoint backing LRO: projects/692586115434/locations/us-central1/indexEndpoints/3295135193374916608/operations/4176750726359285760
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:MatchingEngineIndexEndpoint created. Resource name: projects/692586115434/locations/us-central1/indexEndpoints/3295135193374916608
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:To use this MatchingEngineIndexEndpoint in another session:
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/692586115434/locations/us-central1/indexEndpoints/3295135193374916608')
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Deploying index Matc

KeyboardInterrupt: 

**Index Object:**

Represents the actual vector database containing your embeddings.
It's where your vector data is stored and organized for efficient similarity search.
You create an Index when you want to add or update the vector data that you want to search through.
The Index object is used for operations like adding or updating vector data.


**Index Endpoint:**

Represents a deployed instance of an Index that can be queried.
It's the access point for performing vector similarity searches on your data.
You deploy an Index to an Endpoint to make it available for serving query requests.
The Index Endpoint object is used for operations like searching for similar vectors or nearest neighbors.

**TLDR:** use the Index object for managing your vector data, and the Index Endpoint object for performing similarity searches on that data.

In [ ]:
# Testing search
print("Deployed Index ID: ", DEPLOYED_INDEX_ID)
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(DEPLOYED_INDEX_ID)
test_embeddings = get_dense_embedding(["How does longest text looks like?"])

# Test query
response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=test_embeddings,
    num_neighbors=2, # change top_k number here
)

# show the result
import numpy as np

for idx, neighbor in enumerate(response[0]):
    id = np.int64(neighbor.id)
    similar = df.query("id == @id", engine="python")
    print(f"{neighbor.distance:.4f} {similar.title.values[0]}")

Deployed Index ID:  vertexai_embedding_ms2_10081832


InvalidArgument: 400 Request contains an invalid argument.

In [ ]:
# wait for a confirmation
input("Press Enter to delete Index Endpoint, Index and Cloud Storage bucket:")

# delete Index Endpoint
my_index_endpoint.undeploy_all()
my_index_endpoint.delete(force=True)

# delete Index
my_index.delete()

# delete Cloud Storage bucket
! gsutil rm -r "{BUCKET_URI}"

In [ ]:
# saving JSONL file of documents_embedded
import json
# save specific fields only
fields_to_save = ["id", "text", "url", "scraped_datetime", "embedding", "original_text_id", "chunk_id"]

# Select only the specified fields
df = pd.DataFrame(documents_embedded)
df = df[fields_to_save]

# Convert scraped_datetime column to ISO format string
if 'scraped_datetime' in df.columns:
    df['scraped_datetime'] = df['scraped_datetime'].apply(
        lambda x: x.isoformat() if isinstance(x, (datetime, pd.Timestamp)) else x
    )

json_documents_embedded_filename = f"documents_embedded_{UID}.json"
# Save to JSONL
df.to_json(json_documents_embedded_filename, orient='records', lines=True)
print(f"{json_documents_embedded_filename} created successfully.")

documents_embedded_10081802.json created successfully.
